In [1]:
import pandas as pd
import numpy as np

In [37]:
import yfinance as yf
df = yf.download(['AAPL', "GOOGL"], start='2023-01-01', progress=False)
df.head(1)

Price        Adj Close              Close                   High             \
Ticker            AAPL      GOOGL    AAPL      GOOGL        AAPL      GOOGL   
Date                                                                          
2023-01-03  124.048042  89.017838  125.07  89.120003  130.899994  91.050003   

Price              Low                   Open                Volume            
Ticker            AAPL      GOOGL        AAPL      GOOGL       AAPL     GOOGL  
Date                                                                           
2023-01-03  124.169998  88.519997  130.279999  89.589996  112117500  28131200

In [2]:
import pandas as pd

data = {
    'DA': [
        '2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04', '2022-01-05',
        '2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04', '2022-01-05'
    ],
    'CODE': [
        'TWSE Index', 'TWSE Index', 'TWSE Index', 'TWSE Index', 'TWSE Index',
        'TWOTCI Index', 'TWOTCI Index', 'TWOTCI Index', 'TWOTCI Index', 'TWOTCI Index'
    ],
    'HI': [
        18500, 18600, 18700, 18800, 18900,
        14500, 14600, 14700, 14800, 14900
    ],
    'LO': [
        18000, 18100, 18200, 18300, 18400,
        14000, 14100, 14200, 14300, 14400
    ],
    'CL': [
        18200, 18350, 18500, 18650, 18800,
        14200, 14350, 14500, 14650, 14800
    ]
}

price_df = pd.DataFrame(data)

# Convert 'DA' to datetime format
price_df['DA'] = pd.to_datetime(price_df['DA'])


In [49]:
price_df

,DA,CODE,HI,LO,CL
0,2022-01-01,TWSE Index,18500,18000,18200
1,2022-01-02,TWSE Index,18600,18100,18350
2,2022-01-03,TWSE Index,18700,18200,18500
3,2022-01-04,TWSE Index,18800,18300,18650
4,2022-01-05,TWSE Index,18900,18400,18800
5,2022-01-01,TWOTCI Index,14500,14000,14200
6,2022-01-02,TWOTCI Index,14600,14100,14350
7,2022-01-03,TWOTCI Index,14700,14200,14500
8,2022-01-04,TWOTCI Index,14800,14300,14650
9,2022-01-05,TWOTCI Index,14900,14400,14800


In [43]:
filtered_df = price_df[price_df['CODE'].isin(['TWSE Index', 'TWOTCI Index'])].copy()
filtered_df['AvgPrice'] = (filtered_df['HI'] + filtered_df['LO']) / 2
filtered_df['Return'] = filtered_df.groupby('CODE')['AvgPrice'].pct_change()

def calculate_beta(df):
    df['Next_CL'] = df.groupby('CODE')['CL'].shift(-1)
    df['Beta'] = df['Next_CL'] / df['CL'] - 1
    return df

beta_df = filtered_df.groupby('CODE').apply(calculate_beta).reset_index(drop=True)
beta_df.index.name = "code"
def calculate_stddev(df):
    df['Return_STDDEV'] = df['Return'].rolling(window=5).std() * 100
    return df

stddev_df = beta_df.groupby('CODE').apply(calculate_stddev).reset_index(drop=True)
stddev_df

,DA,CODE,HI,LO,CL,AvgPrice,Return,Next_CL,Beta,Return_STDDEV
0,2022-01-01,TWOTCI Index,14500,14000,14200,14250.0,NaN,14350.0,0.010563,NaN
1,2022-01-02,TWOTCI Index,14600,14100,14350,14350.0,0.007018,14500.0,0.010453,NaN
2,2022-01-03,TWOTCI Index,14700,14200,14500,14450.0,0.006969,14650.0,0.010345,NaN
3,2022-01-04,TWOTCI Index,14800,14300,14650,14550.0,0.006920,14800.0,0.010239,NaN
4,2022-01-05,TWOTCI Index,14900,14400,14800,14650.0,0.006873,NaN,NaN,NaN
5,2022-01-01,TWSE Index,18500,18000,18200,18250.0,NaN,18350.0,0.008242,NaN
6,2022-01-02,TWSE Index,18600,18100,18350,18350.0,0.005479,18500.0,0.008174,NaN
7,2022-01-03,TWSE Index,18700,18200,18500,18450.0,0.005450,18650.0,0.008108,NaN
8,2022-01-04,TWSE Index,18800,18300,18650,18550.0,0.005420,18800.0,0.008043,NaN
9,2022-01-05,TWSE Index,18900,18400,18800,18650.0,0.005391,NaN,NaN,NaN


In [4]:
import pandas as pd
import numpy as np

def calculate_stddev(df):
    df['Return_STDDEV'] = df['Return'].rolling(window=5).std() * 100
    return df

# Apply the function to calculate the standard deviation of returns
stddev_df = beta_df.groupby('CODE').apply(calculate_stddev)

# Calculate the moving averages for the last 20 and 60 days
def calculate_moving_averages(df):
    df['MA20'] = df['CL'].rolling(window=20).mean()
    df['MA60'] = df['CL'].rolling(window=60).mean()
    return df

# Apply the function to calculate moving averages
ma_df = stddev_df.groupby('CODE').apply(calculate_moving_averages)

# Filter the data for 'TWSE Index' only and with date greater than or equal to a specified date
start_date = '2022-01-01'  # replace with your actual start date
final_df = ma_df[(ma_df['CODE'] == 'TWSE Index') & (ma_df['DA'] >= start_date)].copy()

# Selecting and renaming columns to match the SQL query output
final_df = final_df[['DA', 'Beta', 'Return_STDDEV', 'MA20', 'MA60', 'CL']]
final_df.rename(columns={'Beta': 'TWSE_BETA', 'Return_STDDEV': 'TWSE_RETURN_STDDEV', 'MA20': 'TWSE_MA20', 'MA60': 'TWSE_MA60'}, inplace=True)

# Calculate TWOTCI metrics for each corresponding date in the final_df
twotci_df = ma_df[ma_df['CODE'] == 'TWOTCI Index'].copy()

def calculate_twotci_metrics(row):
    date = row['DA']
    twotci_today = twotci_df[twotci_df['DA'] == date]
    twotci_next = twotci_df[twotci_df['DA'] > date].sort_values(by='DA').head(1)

    if not twotci_today.empty and not twotci_next.empty:
        twotci_beta = twotci_next['CL'].values[0] / twotci_today['CL'].values[0] - 1
    else:
        twotci_beta = np.nan
    
    twotci_return_stddev = twotci_df[(twotci_df['DA'] >= (pd.to_datetime(date) - pd.Timedelta(days=5))) & (twotci_df['DA'] <= date)]['Return'].std() * 100
    twotci_ma20 = twotci_df[twotci_df['DA'] <= date].sort_values(by='DA', ascending=False).head(20)['CL'].mean()
    twotci_ma60 = twotci_df[twotci_df['DA'] <= date].sort_values(by='DA', ascending=False).head(60)['CL'].mean()
    twotci_cl = twotci_today['CL'].values[0] if not twotci_today.empty else np.nan
    
    return pd.Series([twotci_beta, twotci_return_stddev, twotci_ma20, twotci_ma60, twotci_cl])

final_df[['TWOTCI_BETA', 'TWOTCI_RETURN_STDDEV', 'TWOTCI_MA20', 'TWOTCI_MA60', 'TWOTCI_CL']] = final_df.apply(calculate_twotci_metrics, axis=1)

# Display the final DataFrame
print(final_df)


ValueError: 'CODE' is both an index level and a column label, which is ambiguous.